CIFAR-10

In [0]:
import numpy as np
import keras
from keras.models import Sequential
from keras import regularizers
from keras.callbacks import LearningRateScheduler
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D
from keras.datasets import cifar10
 
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    if epoch > 100:
        lrate = 0.0003
    return lrate
 
#Loading data
(xTrain, yTrain), (xTest, yTest) = cifar10.load_data()
xTrain = xTrain.astype('float32')
xTest = xTest.astype('float32')
classes = 10
yTrain = np_utils.to_categorical(yTrain,classes)
yTest = np_utils.to_categorical(yTest,classes)

mean = np.mean(xTrain,axis=(0,1,2,3))
std = np.std(xTrain,axis=(0,1,2,3))
xTrain = (xTrain-mean)/(std+1e-7)
xTest = (xTest-mean)/(std+1e-7)

model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.0001), input_shape=xTrain.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.0001)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.0001)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.0001)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.0001)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.0001)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(classes, activation='softmax'))
 
model.summary()
 
data = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    )

data.fit(xTrain)

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.rmsprop(lr=0.001,decay=1e-6), metrics=['accuracy'])
results = model.fit_generator(data.flow(xTrain, yTrain, batch_size=64),\
                    steps_per_epoch=xTrain.shape[0] // 64,epochs=110,\
                    verbose=1,validation_data=(xTest,yTest),callbacks=[LearningRateScheduler(lr_schedule)])
 
#testing
final = model.evaluate(xTest, yTest, batch_size=128, verbose=1)

Using TensorFlow backend.


170500096/170498071 [==============================] - 26s 0us/step
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
___________________

In [0]:
print('\nTest result: %.3f loss: %.3f' % (final[1]*100,final[0]))


Test result: 87.530 loss: 0.494


In [0]:
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
accuracy = final.history['acc']
val_accuracy = final.history['val_acc']
loss = final.history['loss']
val_loss = final.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()